In [9]:
from prompting.datasets.wiki import WikiDataset
dataset = WikiDataset()

samples = []
for i in range(1000):
    samples.append(dataset.random())

/workspace/prompting/.venv/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /workspace/prompting/.venv/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
2024-10-13 18:31:54.080 | DEBUG    | prompting.datasets.wiki:_get_page:31 - DisambiguationError loading page 'WBBZ': "WBBZ" may refer to: 
WBBZ (AM)
WBBZ-TV
2024-10-13 18:37:45.624 | DEBUG    | prompting.datasets.wiki:_get_page:31 - DisambiguationError loading page 'Efflux': "Efflux" may refer to: 
Efflux (microbiology)
e-

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3                                                                                              │
│   4 samples = []                                                                                 │
│   5 for i in range(1000):                                                                        │
│ ❱ 6 │   samples.append(dataset.random())                                                         │
│   7                                                                                              │
│                                                                                                  │
│ /workspace/prompting/prompting/datasets/wiki.py:207 in random                                    │
│                                                                                                  │
│   204 │   def random(self, pages=10) -> dict:                                                    │
│   205 │   │   titles = _get_random_titles(pages=pages)                                           │
│   206 │   │   for title in titles[: self.max_tries]:                                             │
│ ❱ 207 │   │   │   if context := self.get(title):                                                 │
│   208 │   │   │   │   return context                                                             │
│   209 │   │   return None                                                                        │
│   210                                                                                            │
│                                                                                                  │
│ /workspace/prompting/prompting/datasets/wiki.py:185 in get                                       │
│                                                                                                  │
│   182 │   │   │   content=section_title,                                                         │
│   183 │   │   │   internal_links=list(filter(lambda x: x not in exclude, page.sections)),        │
│   184 │   │   │   external_links=most_relevant_links(page, num_links=self.max_links),            │
│ ❱ 185 │   │   │   tags=filter_categories(page.categories, exclude=self.EXCLUDE_CATEGORIES),      │
│   186 │   │   │   source="Wikipedia",                                                            │
│   187 │   │   │   extra={                                                                        │
│   188 │   │   │   │   "url": page.url,                                                           │
│                                                                                                  │
│ /workspace/prompting/.venv/lib/python3.11/site-packages/wikipedia/wikipedia.py:626 in categories │
│                                                                                                  │
│   623 │                                                                                          │
│   624 │   if not getattr(self, '_categories', False):                                            │
│   625 │     self._categories = [re.sub(r'^Category:', '', x) for x in                            │
│ ❱ 626 │   │   [link['title']                                                                     │
│   627 │   │   for link in self.__continued_query({                                               │
│   628 │   │     'prop': 'categories',                                                            │
│   629 │   │     'cllimit': 'max'                                                                 │
│                                                                                                  │
│ /workspace/prompting/.venv/lib/python3.11/site-packages/wik

In [10]:
print(len(samples))

402


In [11]:
# Perform the analysis

import re
from typing import List
from collections import defaultdict
import matplotlib.pyplot as plt


def get_length_index(answers: List[str], correct: int) -> int:
    """Returns the index of the correct answer once the answers are sorted by string length."""
    assert len(answers) == 4
    sorted_by_length = sorted(answers, key=len)
    return sorted_by_length.index(answers[correct])


def plot_results(prompt_name, questions, answers):
    answer_map = {
        "A": 0,
        "B": 1,
        "C": 2,
        "D": 3
    }
    
    # Convert the answer into an index.
    correct_answers = [answer_map[a] for a in answers]

    r = "\[Input Question\]\s+(.*)\s+A\. (.*)\s+B\. (.*)\s+C\. (.*)\s+D\. (.*)"
    regex_trim = "\[Input Question\]\s+([\s\S]*)Answer:.*"

    length_counts = defaultdict(list)

    for question, correct in zip(questions, correct_answers):
        match = re.search(r, question)
        if not match:
            print(f"WTF: {question}")
            continue
        
        trimmed_qa = re.search(regex_trim, question).groups()[0]
        four_answers = match.groups()[1:]
        length_counts[get_length_index(four_answers, correct)].append((trimmed_qa, correct))

    x_and_y = [(index, len(items)) for index, items in length_counts.items()]
    x, y = zip(*x_and_y)

    plt.xticks(ticks=range(3))
    plt.bar(x, y)
    plt.xlabel("Index of correct ans by answer length")
    plt.ylabel("Frequency")
    plt.title(prompt_name)
    plt.show()

In [ ]:
from prompting import settings
import prompting
settings.settings = settings.Settings(mode="validator")
settings = settings.settings
from prompting.tasks.multi_choice import MultiChoiceTask
from prompting.tasks.multi_choice import QUERY_SYSTEM_PROMPT
from prompting.tasks.multi_choice import QUERY_PROMPT_TEMPLATE


def create_multi_choice_question(sample, system_prompt, user_prompt):
    task = MultiChoiceTask()
    _ = task.make_query(sample, system_prompt, user_prompt)
    return (task.query, task.reference)

USER_PROMPT_WITH_LENGTH_PROMPT = """\
Create a multiple choice quiz based on the following context source from {source} about {title}. All answers must be of similar length, tone and style.

[Input Context]
{context}
"""

MINIMAL_USER_PROMPT = """
The following context from {source} is about {title}.

[Input Context]
{context}
"""

SYSTEM_PROMPT_WITH_LENGTH_PROMPT = """You are a multiple choice quiz-generating expert.
Based on the input context, you must generate the question, exactly 4 possible answers (A, B, C, D), and the correct answer letter.
All answers must be of similar length, tone and style.

[Example 1]
{
    "question": "What is the capital of Texas?",
    "A": "Paris",
    "B": "London",
    "C": "Austin",
    "D": "Houston",
    "answer": "C"
}

[Example 2]
{
    "question": "Which of the following best describes the primary driving force behind protein folding?",
    "A": "Covalent bond formation between amino acids",
    "B": "Hydrophobic interactions between nonpolar side chains",
    "C": "Hydrogen bonds between the protein backbone and side chains",
    "D": "Ionic interactions between charged side chains",
    "answer": "B"
}
"""

SYSTEM_PROMPT_MORE_EXAMPLES = """You are a multiple choice quiz-generating expert.
Based on the input context, you must generate the question, exactly 4 possible answers (A, B, C, D), and the correct answer letter.
All answers must be of similar length, tone and style.

[Example 1]
{
    "question": "Which of the following is not an element of the redistribution-with-growth policy approach?",
    "A": "minimum wage legislation",
    "B": "land reform",
    "C": "progressive taxation",
    "D": "increased access to education",
    "answer": "A"
}

[Example 2]
{
    "question": "Which of the following best describes the primary driving force behind protein folding?",
    "A": "Covalent bond formation between amino acids",
    "B": "Hydrophobic interactions between nonpolar side chains",
    "C": "Hydrogen bonds between the protein backbone and side chains",
    "D": "Ionic interactions between charged side chains",
    "answer": "B"
}

[Example 3]
{
    "question": "What is the capital of Texas?",
    "A": "Paris",
    "B": "London",
    "C": "Austin",
    "D": "Houston",
    "answer": "C"
}

[Example 4]
{
    "question": "What interior discipline must be adopted to achieve spiritual liberation within Sikhism?",
    "A": "Remembering the Divine Name",
    "B": "Meditating on the sacred hymns",
    "C": "Remembering that death is inevitable",
    "D": "Meditating on the goodness of the created world",
    "answer": "A"
}
"""

SYSTEM_PROMPT_NEW_FORMAT = """Given the following input context, create a multiple-choice question based on the information provided. The question must have one correct answer and three incorrect answers.
Ensure the following:

1. The correct answer is derived from the input context.
2. All answer choices have roughly the same character length.
3. The output format must match the example's output format.

[Example 1]
{
    "question": "Which of the following is not an element of the redistribution-with-growth policy approach?",
    "A": "minimum wage legislation",
    "B": "land reform",
    "C": "progressive taxation",
    "D": "increased access to education",
    "answer": "A"
}

[Example 2]
{
    "question": "Which of the following best describes the primary driving force behind protein folding?",
    "A": "Covalent bond formation between amino acids",
    "B": "Hydrophobic interactions between nonpolar side chains",
    "C": "Hydrogen bonds between the protein backbone and side chains",
    "D": "Ionic interactions between charged side chains",
    "answer": "B"
}

[Example 3]
{
    "question": "What is the capital of Texas?",
    "A": "Paris",
    "B": "London",
    "C": "Austin",
    "D": "Houston",
    "answer": "C"
}

[Example 4]
{
    "question": "What interior discipline must be adopted to achieve spiritual liberation within Sikhism?",
    "A": "Remembering the Divine Name",
    "B": "Meditating on the sacred hymns",
    "C": "Remembering that death is inevitable",
    "D": "Meditating on the goodness of the created world",
    "answer": "A"
}
"""

ANSWER_MUST_BE_SHORTEST = """Given the following input context, create a multiple-choice question based on the information provided. The question must have one correct answer and three incorrect answers.
Ensure the following:

1. The correct answer is derived from the input context.
2. The correct answer must be the shortest answer by count of characters.
3. The output format must match the example's output format.

[Example 1]
{
    "question": "Which of the following is not an element of the redistribution-with-growth policy approach?",
    "A": "minimum wage legislation",
    "B": "land reform",
    "C": "progressive taxation",
    "D": "increased access to education",
    "answer": "A"
}

[Example 2]
{
    "question": "Which of the following best describes the primary driving force behind protein folding?",
    "A": "Covalent bond formation between amino acids",
    "B": "Hydrophobic interactions between nonpolar side chains",
    "C": "Hydrogen bonds between the protein backbone and side chains",
    "D": "Ionic interactions between charged side chains",
    "answer": "B"
}

[Example 3]
{
    "question": "What is the capital of Texas?",
    "A": "Paris",
    "B": "London",
    "C": "Austin",
    "D": "Houston",
    "answer": "C"
}

[Example 4]
{
    "question": "What interior discipline must be adopted to achieve spiritual liberation within Sikhism?",
    "A": "Remembering the Divine Name",
    "B": "Meditating on the sacred hymns",
    "C": "Remembering that death is inevitable",
    "D": "Meditating on the goodness of the created world",
    "answer": "A"
}
"""

INCORRECT_MUST_BE_PLAUSIBLE = """Given the following input context, create a multiple-choice question based on the information provided. The question must have one correct answer and three incorrect answers.
You must also follow the following rules:

1. The correct answer is derived from the input context.
2. The incorrect answers must be of similar style and tone to the correct answer.
3. The incorrect answers must contain at least as many characters as the correct answer.
4. The output format must match the example's output format.

[Example 1]
{
    "question": "Which of the following is not an element of the redistribution-with-growth policy approach?",
    "A": "minimum wage legislation",
    "B": "land reform",
    "C": "progressive taxation",
    "D": "increased access to education",
    "answer": "A"
}

[Example 2]
{
    "question": "Which of the following best describes the primary driving force behind protein folding?",
    "A": "Covalent bond formation between amino acids",
    "B": "Hydrophobic interactions between nonpolar side chains",
    "C": "Hydrogen bonds between the protein backbone and side chains",
    "D": "Ionic interactions between charged side chains",
    "answer": "B"
}

[Example 3]
{
    "question": "What is the capital of Texas?",
    "A": "Paris",
    "B": "London",
    "C": "Austin",
    "D": "Houston",
    "answer": "C"
}

[Example 4]
{
    "question": "What interior discipline must be adopted to achieve spiritual liberation within Sikhism?",
    "A": "Remembering the Divine Name",
    "B": "Meditating on the sacred hymns",
    "C": "Remembering that death is inevitable",
    "D": "Meditating on the goodness of the created world",
    "answer": "A"
}

[Example 5]
{
    "question": "The owners of independent gasoline stations in a community meet secretly and agree to keep their prices high to protect their profitability. If their plan is discovered and they are charged with a crime, they could be",
    "A": "found guilty under the Sherman Antitrust Act and face prison time",
    "B": "found guilty under the Magnuson-Moss Warranty Act and face large fines",
    "C": "found guilty under the Adamson Act and face prison time and fines",
    "D": "found not guilty because retailers are allowed to set their own price levels",
    "answer": "A"
}
"""

prompts = {
    # "default": (QUERY_SYSTEM_PROMPT, QUERY_PROMPT_TEMPLATE),
    # "length_instructions": (QUERY_SYSTEM_PROMPT, USER_PROMPT_WITH_LENGTH_PROMPT),
    # "more_examples": (SYSTEM_PROMPT_MORE_EXAMPLES, QUERY_PROMPT_TEMPLATE),
    # "more_examples_and_length": (SYSTEM_PROMPT_MORE_EXAMPLES, USER_PROMPT_WITH_LENGTH_PROMPT),
    # "formatted_instructions": (SYSTEM_PROMPT_NEW_FORMAT, QUERY_PROMPT_TEMPLATE),
    # "formatted_instructions_simple_user": (SYSTEM_PROMPT_NEW_FORMAT, MINIMAL_USER_PROMPT),
    # "formatted_instructions_with_length_user_prompt": (SYSTEM_PROMPT_NEW_FORMAT, USER_PROMPT_WITH_LENGTH_PROMPT),
    # "make_answer_short": (ANSWER_MUST_BE_SHORTEST,QUERY_PROMPT_TEMPLATE)
    "more_rules": (INCORRECT_MUST_BE_PLAUSIBLE, QUERY_PROMPT_TEMPLATE)
}

for name, (system_prompt, user_prompt) in prompts.items():
    print(f"Performing analysis on {name}")
    questions = []
    answers = []
    for sample in samples:
        try:
            question, answer = create_multi_choice_question(sample, system_prompt, user_prompt)
            questions.append(question)
            answers.append(answer)
        except Exception as e:
            print(f"Failed to create sample for {name}")      
            
    plot_results(name, questions, answers)  


Performing analysis on more_rules


2024-10-14 00:34:21.736 | INFO     | prompting.tasks.base_task:generate_query:102 - 🤖 Generating query...

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.57s/it, est. speed input: 179.10 toks/s, output: 26.07 toks/s]
2024-10-14 00:34:25.317 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


Here is a multiple-choice question based on the input context:

{
    "question": "According to FCC records, when was WBBZ first licensed?",
    "A": "January 1, 1923",
    "B": "September 9, 1925",
    "C": "December 31, 1929",
    "D": "March 15, 1932",
    "answer": "B"
}
2024-10-14 00:34:25.318 | INFO     | prompting.tasks.base_task:generate_query:102 - 🤖 Generating query...

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.87s/it, est. speed input: 170.69 toks/s, output: 26.12 toks/s]
2024-10-14 00:34:29.194 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


{
    "question": "Who were Joha

Failed to create sample for more_rules


2024-10-14 00:35:49.375 | INFO     | prompting.tasks.base_task:generate_query:102 - 🤖 Generating query...

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.45s/it, est. speed input: 191.26 toks/s, output: 25.50 toks/s]
2024-10-14 00:35:52.833 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


{
    "question": "What award did Morteza Najafi win for his work on the 2020 Iranian drama film Tooman?",
    "A": "Golden Lion for Best Director",
    "B": "Crystal Simorgh for Best Cinematography",
    "C": "Silver Bear for Best Screenplay",
    "D": "Golden Globe for Best Original Score",
    "answer": "B"
}
2024-10-14 00:35:52.834 | INFO     | prompting.tasks.base_task:generate_query:102 - 🤖 Generating query...

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.96s/it, est. speed input: 223.17 toks/s, output: 24.68 toks/s]
2024-10-14 00:35:55.798 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following outp